## Rede Neural Multilayer Perceptron para reconhecimento de padrão:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter, LogLocator

import numpy as np
import pandas as pd
import os

import random
from math import cos, tanh
from time import sleep

from ipywidgets import *
import ipywidgets as widgets
import re

from IPython.display import display, HTML

########## entradas dos parametros ##########
style = {'description_width': 'initial'}

print('\n\nTaxa de aprendizagem:')
taxa_aprendizagem = widgets.FloatSlider(
    value=0.01,
    min=0.01,
    max=1,
    step=0.01,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
display(taxa_aprendizagem)

print('\n\nQuantidade de neurônios da camada intermediária:')
qnt_neuronios = widgets.IntSlider(
    value=30,
    min=1,
    max=100,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(qnt_neuronios)

print('\n\nErro tolerado:')
erro = widgets.FloatSlider(
    value=0.06,min=0,max=1,step=0.001,
    continuous_update=False,
    readout=True, style=style,
    readout_format='.5f',
)
display(erro)

class RedeMLP():
    def __init__(self, conjunto_entradas, targets, taxa_aprendizagem, parada, ciclos_max, 
                 erro_tolerado, qnt_neuronios, **kwargs):
        self.numero_entradas = 256 #tamanho da imagem
        self.numero_saidas = 16 #0-9
        self.conjunto_entradas = np.array(conjunto_entradas)
        self.targets = np.array(targets)
        self.taxa_aprendizagem = taxa_aprendizagem
        self.taxa_aprendizagem_inicial = taxa_aprendizagem
        self.ciclos = 0
        self.ciclos_max = ciclos_max
        self.parada = parada
        #Quantidade de neuronios seleciondos para camada intermediária
        self.qnt_neuronios_intermediarios = qnt_neuronios
        self.erro_tolerado = erro_tolerado
        #Pesos das ligações entre as camadas inicial e intermediária; intermediária e saída
        self.pesos_ini_inter, self.pesos_inter_saida = self.__inicializa_pesos()
        #Iniciliza os bias da camada intermediária
        self.bias_intermediario = np.asmatrix(np.array([random.uniform(-0.5,0.5) for saida in range(qnt_neuronios)]))
        #Inicializa o bias da camada de saida
        self.bias_saida = np.asmatrix(np.array([random.uniform(-0.5,0.5) for i in range(self.numero_saidas)]))
        self.erro = 0
        
    def set_pesos_ini_inter(self, pesos):
        self.pesos_ini_inter = pesos
        
    def set_pesos_inter_saida(self, pesos):
        self.pesos_inter_saida = pesos
        
    def set_bias_intermediario(self, bias):
        self.bias_intermediario = bias
        
    def set_bias_saida(self, bias):
        self.bias_saida = bias
        
    def __inicializa_pesos(self):
        pesos_ini_inter = []
        pesos_inter_saida = []
        
        #mxn = onde m é o número de entradas e n é o número de neurônios da camada intermediária
        for i in range(self.numero_entradas):
            peso_aux = []
            for j in range(self.qnt_neuronios_intermediarios):
                peso_aux.append(random.uniform(-0.5,0.5))
            pesos_ini_inter.append(peso_aux)
        
        #nxk onde n é o número de neurônios da camada intermediária e k é o número de saídas
        for i in range(self.qnt_neuronios_intermediarios):
            peso_aux = []
            for i in range(self.numero_saidas):
                peso_aux.append(random.uniform(-0.5,0.5))
            pesos_inter_saida.append(peso_aux)
            
        #print("Tam peso ini_inter", len(pesos_ini_inter[0]))
        #print("Tam peso inter_saida", len(pesos_inter_saida[0]))
        return np.array(pesos_ini_inter), np.array(pesos_inter_saida)
    
    def __f_ativacao(self, matriz):
        #print("ZIN", zin)
        out = []
        dim = matriz.shape #dimensão da matriz
        for i in range(dim[0]):
            aux = []
            for j in range(dim[1]):
                aux.append(tanh(matriz.item(i,j)))
            out.append(aux)
                
        return np.matrix(np.array(out))
    
    def __calcula_saida__(self, saida):
        dists_euclidiana = []
        for i in range(10):
            calculo = np.sum(np.power(np.power((self.targets[i]-saida),2),0.5))
            dists_euclidiana.append(calculo)
        menor_valor = np.amin(dists_euclidiana)
        id_menor_valor = dists_euclidiana.index(menor_valor)
        return np.asmatrix(self.targets[id_menor_valor]);
    
    def __normaliza_saida__(self, saida):
        saida[saida >= 0.5] = 1
        saida[saida < 0.5] = 0
        return saida
        
    
    def treinamento(self):
        treinada = False
        erros = []
        
        pesos_inter_saida_anterior = []
        bias_saida_anterior = []
        pesos_ini_inter_anterior = []
        bias_intermediario_anterior = []
        
        while not treinada:
            i = 0 
            self.erro = 0
            
            for entrada_i in self.conjunto_entradas:
                ####################Etapa de Forward#################################
                ##############Camada inicial para intermediária
                #resultado = 1xn, onde n é o número de neurônios da camada intermediária
                
                zin = (np.asmatrix(entrada_i) @ self.pesos_ini_inter) + self.bias_intermediario
                z = self.__f_ativacao(zin)

                ##############Camada intermediária para saída
                yin = (z @ self.pesos_inter_saida) + self.bias_saida
                y = self.__f_ativacao(yin)
                #y = self.__calcula_saida__(y)
                #print("y", y)
                
                
                #####################Etapa de backward
                #Cálculo do erro
                self.erro += 0.5 * np.sum(np.power((self.targets[i] - y), 2))
                
                #########cálculo dos parâmetros para correção dos erros da camada intermediária para camada de saída
                #Calculo dos parâmetros para correção dos pesos e do bias
                #np.multiply para se fazer uma multiplicação item por item
                deltak = np.multiply((self.targets[i] - y).T,(1 + y).T)
                deltak = np.multiply(deltak, (1 - y).T)
                
                #Parâmetro para correção dos pesos
                delta_pesos_inter_saida = deltak @ z
                delta_pesos_inter_saida = self.taxa_aprendizagem * delta_pesos_inter_saida
                
                #Parâmetro para correção do bias
                delta_bias_saida = np.multiply(self.taxa_aprendizagem, deltak)
                
                
                #########cálculo dos parâmetros para correção dos erros da camada entrada para camada de intermediária
                #Calculo dos parâmetros para correção dos pesos e do bias
                delta_in = deltak.T @ self.pesos_inter_saida.T
                
                
                deltaj = np.multiply(delta_in, (1+z))
                deltaj = np.multiply(deltaj, (1-z))
                
                #Parâmetro para correção dos pesos
                delta_pesos_ini_inter = deltaj.T @ np.asmatrix(entrada_i)
                delta_pesos_ini_inter = np.multiply(self.taxa_aprendizagem, delta_pesos_ini_inter)
                
                delta_bias_intermediario = np.multiply(self.taxa_aprendizagem, deltaj)

                if (self.ciclos == 0) and (i == 0):
                    pesos_inter_saida_anterior = self.pesos_inter_saida
                    bias_saida_anterior = self.bias_saida
                    pesos_ini_inter_anterior = self.pesos_ini_inter
                    bias_intermediario_anterior = self.bias_intermediario
                    #Correção dos pesos da camada intermediária para camada de saída e bias da camada de saída
                    self.pesos_inter_saida = self.pesos_inter_saida + delta_pesos_inter_saida.T 
                    self.bias_saida = self.bias_saida + delta_bias_saida.T
                    #Correção dos pesos da camada intermediária para camada de saída e bias da camada de intermediária
                    self.pesos_ini_inter = self.pesos_ini_inter + delta_pesos_ini_inter.T
                    self.bias_intermediario = self.bias_intermediario + delta_bias_intermediario
                else:
                    const_momentum = random.uniform(0.15, 0.75)
                    termo_momentum = const_momentum * (self.pesos_inter_saida - pesos_inter_saida_anterior)
                    pesos_inter_saida_anterior = self.pesos_inter_saida
                    self.pesos_inter_saida = self.pesos_inter_saida + delta_pesos_inter_saida.T + termo_momentum
                    
                    termo_momentum = const_momentum * (self.bias_saida - bias_saida_anterior)
                    bias_saida_anterior = self.bias_saida
                    self.bias_saida = self.bias_saida + delta_bias_saida.T + termo_momentum
                    
                    termo_momentum = const_momentum * (self.pesos_ini_inter - pesos_ini_inter_anterior)
                    pesos_ini_inter_anterior = self.pesos_ini_inter
                    self.pesos_ini_inter = self.pesos_ini_inter + delta_pesos_ini_inter.T + termo_momentum
                    
                    termo_momentum = const_momentum * (self.bias_intermediario - bias_intermediario_anterior)
                    bias_intermediario_anterior = self.bias_intermediario
                    self.bias_intermediario = self.bias_intermediario + delta_bias_intermediario + termo_momentum
                
                i+=1

            self.erro = (1/900)*self.erro
            erros.append(self.erro)
            print('erro', self.erro)
            if (self.parada == 'ciclos' and self.ciclos >= self.ciclos_max) \
                or (self.parada == 'erro' and self.erro <= self.erro_tolerado):
                #print('erro final: ', self.erro)
                treinada = True
            else:
                self.ciclos += 1
                
        return list(range(0, self.ciclos+1)), erros
    
    def operacao(self, conjunto_testes, targets_testes):
        acertos = 0
        i = 0
        for entrada_i in conjunto_testes:
            zin = (np.asmatrix(entrada_i) @ self.pesos_ini_inter) + self.bias_intermediario
            z = self.__f_ativacao(zin)

            yin = (z @ self.pesos_inter_saida) + self.bias_saida
            y = self.__f_ativacao(yin)
            #y = self.__normaliza_saida__(y)
            y = self.__calcula_saida__(y)
            if(np.array_equal(np.asmatrix(y), np.asmatrix(targets_testes[i]))):
                acertos += 1
            i+=1

        return acertos

##### definindo as entradas a partir do arquivo digitostreinamento900.txt #####
def get_inputs_from_txt():
    inputs = []
    file = open('digitos/digitostreinamento900.txt', 'r')
    for line in file :
        line = line.split(" ")
        ipt = []
        for element in line:
            if element != "":
                ipt.append(float(element))
        inputs.append(ipt)
    file.close()
    return np.array(inputs)
##############################################


def get_inputs_test_from_txt():
    path_to_folder = 'C:\\Users\\Marco Antônio\\Documents\\IA\\Trabalhos\\trabalhos-ia\\trab9\\teste'
    inputs = []
    for i in os.listdir(path_to_folder):
        file = open(path_to_folder + '\\' + i, 'r')
        for line in file :
            line = line.split(" ")
            ipt = []
            for element in line:
                if element != "":
                    ipt.append(float(element))
            inputs.append(ipt)
        file.close()
    return np.array(inputs)

def get_obvs(k, m):
    semente = np.array([[1]])
    obvs = []
    for i in range(k):
        aux_obv = []
        for j in range(2**i):
            comb1 = np.array(np.concatenate((semente[j], semente[j])))
            comb2 = np.array(np.concatenate((semente[j], -1*semente[j])))
            aux_obv.append(np.array(comb1))
            aux_obv.append(np.array(comb2))
        semente = aux_obv
        obvs = aux_obv
        
    targets = []
    for i in range(90):
        for j in range(10):
            targets.append(obvs[j])
        
    return np.array(targets)

def get_obvs_teste(k, m):
    semente = np.array([[1]])
    obvs = []
    for i in range(k):
        aux_obv = []
        for j in range(2**i):
            comb1 = np.array(np.concatenate((semente[j], semente[j])))
            comb2 = np.array(np.concatenate((semente[j], -1*semente[j])))
            aux_obv.append(np.array(comb1))
            aux_obv.append(np.array(comb2))
        semente = aux_obv
        obvs = aux_obv
        
    targets = []
    for i in range(10):
        for j in range(45):
            targets.append(obvs[i])
        
    return np.array(targets)

def get_targets():
    targets_aux = pd.read_csv('targets.csv').values
    targets = []
    for i in range(90):
        for j in range(10):
            targets.append(list(targets_aux[j]))
    return np.array(targets)

def get_targets_teste():
    targets_aux = pd.read_csv('targets.csv').values
    targets = []
    for i in range(10):
        for j in range(45):
            targets.append(list(targets_aux[i]))
    return np.array(targets)

def processar():
    conjunto_entradas = get_inputs_from_txt()
    #targets = get_targets()
    targets = get_obvs(4,1)
    mlp = RedeMLP(
        conjunto_entradas = conjunto_entradas, 
        targets = targets, 
        taxa_aprendizagem = taxa_aprendizagem.value,
        parada = 'erro',
        ciclos_max = 0,
        erro_tolerado = erro.value,
        qnt_neuronios = qnt_neuronios.value
    )

    #print("Entradas: ", mlp.conjunto_entradas)
    lista_ciclos, lista_erros = mlp.treinamento() 
    conjunto_testes = get_inputs_test_from_txt()
    targets_testes = get_obvs_teste(4, 1)
    #targets_testes = get_targets_teste()
    acertos = mlp.operacao(conjunto_testes, targets_testes)
    np.savetxt('vij.txt',mlp.pesos_ini_inter, delimiter=",")
    np.savetxt('v0j.txt',mlp.bias_intermediario, delimiter=",")
    np.savetxt('wij.txt',mlp.pesos_inter_saida, delimiter=",")
    np.savetxt('w0j.txt',mlp.bias_saida, delimiter=",")
    print('acertos:', acertos)
    print('ciclos:', mlp.ciclos)
    print('taxa_acertos: {:f}'.format((acertos/450) *100))

    
    
def read_matrix_in_txt(path):
    file = open(path)
    matrix = []
    for line in file:
        matrix.append(list(map(float, line.split(','))))
    return np.asmatrix(matrix)
    
def executar_mlp():
    conjunto_testes = get_inputs_test_from_txt()
    targets_testes = get_obvs_teste(4, 1)
    mlp = RedeMLP(
        conjunto_entradas = conjunto_testes, 
        targets = targets_testes, 
        taxa_aprendizagem = 0.01,
        parada = 'erro',
        ciclos_max = 0,
        erro_tolerado = 0,
        qnt_neuronios = 30
    )
    
    v0j = read_matrix_in_txt('v0j.txt')
    vij = read_matrix_in_txt('vij.txt')
    w0j = read_matrix_in_txt('w0j.txt')
    wij = read_matrix_in_txt('wij.txt')
    
    mlp.set_pesos_ini_inter(vij)
    mlp.set_bias_intermediario(v0j)
    mlp.set_pesos_inter_saida(wij)
    mlp.set_bias_saida(w0j)
    print(mlp.bias_intermediario)
        
    acertos = mlp.operacao(conjunto_testes, targets_testes)
    print("Acertos da rede: ", acertos)
    print("Taxa de acerto: ", acertos * 100/450)
    
    
widgets.interact_manual.opts['manual_name'] = 'Treinar a rede' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao



Taxa de aprendizagem:


FloatSlider(value=0.01, continuous_update=False, max=1.0, min=0.01, readout_format='.3f', step=0.01)



Quantidade de neurônios da camada intermediária:


IntSlider(value=30, continuous_update=False, min=1)



Erro tolerado:


FloatSlider(value=0.06, continuous_update=False, max=1.0, readout_format='.5f', step=0.001, style=SliderStyle(…

interactive(children=(Button(description='Treinar a rede', style=ButtonStyle()), Output()), _dom_classes=('wid…